In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import pandas_profiling

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso, Ridge, LassoCV
from sklearn.metrics import mean_squared_error as mse

# all my custom transformers are put into separate package with docstrings (help(ClassName))
from custom_transformers import MyDropColumns, MyQualityEncoder, MyOtherOrdinalEncoder, MyBinaryEncoder, MySimpleImputer, MyLog1pTransformer
from custom_transformers import MyValueAddedFeatures, MyTimeBasedFeatures, MyQualityFeatures, MyRoomsFeatures, MySpaceBasedFeatures, MyDummyFeatures

# 0. Business understanding

The goal is to predict sales price of the property in Ames, Iowa. To do that, we are provided with a dataset containing information about each property in 79 different dimensions. The short description of each variable is given in the document called "data_description.txt". After going through this document and trying to understand each variable and how it might affect the price (based on research on Google), I clustered all variables into following 12 categories:

1. <u><b>location</b></u>: MSZoning (type of zoning implies different laws), Neighborhood (physical location: should strongly affect the price), Condition1 and Condition2 (close to parks is good, close to railroads is not), 
2. <u><b>lot</b></u>: LotFrontage (wider frontage is usually better), LotArea (bigger area is usually better), LotShape (each has benefits and drawbacks), LandContour (flatness of the property: unlikely affecting the price), LandSlope (slope of the property: unlikely affecting the price), LotConfig (each has drawbacks and benefits)
3. <u><b>public infrastructure</b></u>: Street (type of road: pavel is better than gravel), Alley (type of alley access: pavel is betten than gravel)
4. <u><b>building's general characteristics</b></u>: BldgType (might affect the price), HouseStyle (might affect the price), YearBuilt and YearRemodAdd (should affect the price), Foundation (type: should affect the price)
5. <u><b>building's quality and conditions (all should strongly affect the price)</b></u>: OverallQual, OverallCond, ExterQual and ExterCond (quality and conditions of exterior material)
6. <u><b>building's external characteristics (all might affect the price)</b></u>: RoofStyle and RoofMatl (type and material of roof), Exterior1st and Exterior2nd (exterior covering on house), MasVnrType and MasVnrArea (type and area of masonry veneer), WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch. 
7. <u><b>building interior characteristics (all should affect the price)</b></u>: 1stFlrSF, 2bdFlrSF, LowQualityFinSf, GrLivArea, FullBath, HalfBath, Bedroom, Kitchen, KitchenQual, TotRmsAvbGrd (number of rooms), Functional (home functionality), Fireplaces, FireplaceQu, 
8. <u><b>basement-related characteristics (if present: affect if useful)</b></u>: BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath, Fireplace in basement
9. <u><b>utilities-related characteristics</b></u>: Utilities (generally more -> better), Heating and HeatingQC (type and quality of heating), CentralAir (presence is good), Electrical (system quality)
8. <u><b>additional characteristics</b></u>: PoolArea (should affect the price), PoolQC, Fence, MiscFeature and MiscVal (should strongly affect the price)
9. <u><b>garage-related characteristics</b></u>: GarageType, GarageYrBlt, GarageCars (size in car capacity), GarageArea (size in square feet),  GarageQual, GarageCond, PavedDrive (paved driveway)
10. <u><b>sales-related info</b></u>: MoSold and YrSold (to determine the age: affects the price), SaleType (type of sale: might affect the price), SaleCondition (condition on sale: might affect the price). 

Generally, with respect to the real estate, there are two main ways to put a price on property, which is used in combination: cost-based and market-based. Cost-based implies that price of the house should be higher than the sum of all costs related to the house: cost of land (hence, lot characteristics should affect), cost of materials used to construct the building, cost of additional features (pool, garage, etc), cost of the building itself (hence, the size, number of rooms, bedrooms, and usefulness of basement matter). Market-based implies that price of the house is affected by market itself: hence location matters and sales-related information is important. 

# 1. Data understanding (EDA)

In [2]:
original_train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
train = original_train.copy()

Generation of report - slow operation, check attached ready HTML5 report instead

In [4]:
# initial_report = train.profile_report(style={'full_width':True}, title='House Prices: Advanced Regression Techniques (initial)')
# initial_report.to_file(output_file='house_prices_report_initial.html')

#### Few conclusions can be made from the report itself:

<u><b>Some features are meaningless</b></u>: Id, Utilities (all except 1 values are the same), MSSubClass (same info included in other variables), Street (all except 6 values are the same), Condition1 and Condition2 (most of values are just "Norm")

<u><b>Features that might need log transformation (to fix skewness)</b></u>: 1stFlrSF, 2ndFlrSF (log1p), LotArea, LotFrontage, GrLivArea, BsmtUnfSF, OverallCond, SalePrice, TotalBsmtSF

<u><b>Features that might need to be transformed into True/False, indicating presence, since most of values are either NAs or have one value</b></u>: 2ndFlrSF, Alley, BldgType, MiscFeature, PoolArea, Fence, MasVnrType, MasVnrArea (drop this column as after binarization it will be the same as MasVnrType)

<u><b>Porch-related features to be analyzed</b></u>: 3SsnPorch, EnclosedPorch, OpenPorchSF, ScreenPorch, WoodDeckSF

<u><b>Categorical features with ordinal values to be encoded</b></u>: BsmtCond, BsmtExposure, BsmtFinType1, BstmFinType2, BsmtQual, ExterCond, ExterQual, FireplaceQu, GarageCond, GarageFinish, GarageQual, HeatingQC, KitchenQual, LotShape, PavedDrive, PoolQC, Electrical, Functional, LandSlope, HouseStyle

<u><b>Categorical features without order (one-hot encoding or label encoding)</b></u>: Exterior1st, Exterior2nd, Foundation, GarageType, Heating, LandContour, LotConfig, MSZoning, Neighborhood, RoofMatl, RoofStyle, SaleCondition, SaleType

# 2. Data preparation / cleaning 

### 2.1 outliers

Instead of removing outliers in train dataset, I will use RobustScaler() to deal with outliers

### 2.1 meaningless features

In [5]:
cols_to_drop = ['MSSubClass', 'Id', 'Utilities', 'Street', 'MasVnrArea', 'Condition1', 'Condition2']
train = MyDropColumns(cols_to_drop).fit_transform(train)

###  2.2 ordinal encoding with imputing missing values

Features related to quality and condition

In [6]:
cols_enc_quality = ['BsmtCond', 'BsmtQual', 'ExterCond', 'ExterQual', 'FireplaceQu', 
                    'GarageCond', 'GarageQual', 'HeatingQC', 'KitchenQual', 'PoolQC']
train = MyQualityEncoder(cols_enc_quality).fit_transform(train)

Other features that can be ordinally encoded

In [7]:
cols_enc_ordinal = ['BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageFinish',
                'LotShape', 'PavedDrive', 'Electrical', 'Functional', 'HouseStyle', 'LandSlope']
train = MyOtherOrdinalEncoder(cols_enc_ordinal).fit_transform(train)

### 2.3 binary encoding with imputing missing values

In [8]:
cols_enc_binary = ['Alley', 'BldgType', 'MiscFeature', 'Fence', 'MasVnrType']
train = MyBinaryEncoder(cols_enc_binary).fit_transform(train)

###  2.4 missing values 

In [9]:
train.isnull().sum()[train.isnull().sum()>0]

LotFrontage    259
GarageType      81
GarageYrBlt     81
dtype: int64

Most likely those missing values indicate absence: absence of Garage and zero Lot Frontage. Let's impute values correspondingly. 

In [10]:
train = MySimpleImputer().fit_transform(train) 

### 2.5 skewness

I will apply Log transformation after adding features. Unless columns are specified, MyLog1pTransformer() transformer applies Log1p on each numerical feature that has skewness > 0.75

# 3. Feature engineering (experimentation)

## 3.1 feature creation

### Ideas for additional features

<u><b>Basement-related features</b></u>:
1. Basement exists: from any of Bsmt features
2. Basement SF value = BsmtFinType1 * BsmtFinSF1 + BsmtFinType2 * BsmtSF2 ---> <i>MyValueAddedFeature(basement=True)</i>
3. Basement Finished share = (1-BsmtUnfSF)/TotalBsmtSF or if TotalBsmtSF = 0 ---> <i>MySpaceBasedFeatures(bsmt_finished=True)</i>
4. Basement Additional Value = Basement SF value * BsmtExposure ---> <i>MyValueAddedFeature(basement_adv=True)</i>
5. Basement evaluation mult = BsmtQual * BsmtCond ---> <i>MyQualityFeatures(basement_mult=True)</i>
6. Basement evaluation sum = BsmtQual + BsmtCond ---> <i>MyQualityFeatures(basement_sum=True)</i>
7. Basement size in comparison = TotalBsmtSF / GrLivArea ---> <i>MySpaceBasedFeatures(bsmt_vs_living=True)</i>

<u><b>Garage-related features</b></u>:
1. Garage value = GarageArea * GarageQual ---> <i>MyValueAddedFeature(garage=True)</i>
2. Garage evaluation mult = GarageQual * GarageCond ---> <i>MyQualityFeatures(garage_mult=True)</i>
3. Garage evaluation sum = GarageQual + GarageCond ---> <i>MyQualityFeatures(garage_sum=True)</i>

<u><b>Bedrooms/bathrooms/kitchen features</b></u>:
1. Total bathrooms = FullBath + 0.5 * HalfBath + BsmtFullBath + 0.5 * BsmtHalfBath ---> <i>MyRoomsFeatures(tot_bath=True)</i>
2. Bathrooms / bedrooms = Total bathrooms / BedroomAbvGr ---> <i>MyRoomsFeatures(bath_vs_bedrooms=True)</i>
3. Bedrooms share space = BedroomAbvGr / GrLivArea ---> <i>MyRoomsFeatures(bedrooms_vs_area=True)</i>
4. Bedrooms share rooms = BedroomAbvGr / TotRmsAbvGrd ---> <i>MyRoomsFeatures(bedrooms_vs_rooms=True)</i>
5. All rooms share space = TotRmsAbvGrd / GrLivArea ---> <i>MyRoomsFeatures(rooms_vs_area=True)</i>
6. Kitchen value = Kitchen * KitchenQual ---> <i>MyValueAddedFeature(kitchen=True)</i>

<u><b>Time/date features</b></u>:
1. Seasonality = season(MoSold) ---> <i>MyTimeBasedFeatures(season=True)</i>
2. Time since construction (house) = YrSold - YearBuilt ---> <i>MyTimeBasedFeatures(since_house_built=True)</i>
3. Time since construction (garage) = YrSold - GarageYrBlt ---> <i>MyTimeBasedFeatures(since_garage_built=True)</i>
4. Time since renovation (house) = YrSold - YearRemodAdd ---> <i>MyTimeBasedFeatures(since_house_remod=True)</i>
5. Remodeled = True if YearRemodAdd is different than YearBuilt ---> <i>MyTimeBasedFeatures(isRemodeled=True)</i>

<u><b>Space/area-related features</b></u>:
1. Total porch area = WoodDeckSF + OpenPorchSF + EnclosedPorch + 3SsnPorch + ScreenPorch ---> <i>MySpaceBasedFeatures(porch=True)</i>
2. Free space left = (LotArea - TotalBsmtSF - GarageArea - PoolArea - Total porch area) / LotArea ---> <i>MySpaceBasedFeatures(lot_left_percent=True)</i>
3. House space share = TotalBsmtSF / LotArea ---> <i>MySpaceBasedFeatures(bsmt_vs_lot=True)</i>

<u><b>Quality-related features</b></u>:
1. High Quality SF = (1-LowQualFinSF)/GrLivArea ---> <i>MyQualityFeatures(high_quality_sf=True)</i>
2. Overall evaluation mult = OverallQual * OverallCond ---> <i>MyQualityFeatures(overall_mult=True)</i>
3. Overall evaluation sum =  OverallQual + OverallCond ---> <i>MyQualityFeatures(overall_sum=True)</i>
4. External material evaluation mult = ExterQual * ExterCond ---> <i>MyQualityFeatures(external_mult=True)</i>
5. External material evaluation sum = ExterQual + ExterCond ---> <i>MyQualityFeatures(external_sum=True)</i>


<u><b>Luxury features</b></u>: 
1. Pool value = PoolArea * PoolQC ---> <i>MyValueAddedFeature(pool=True)</i>
2. Fireplace value = Fireplaces * FireplaceQu ---> <i>MyValueAddedFeature(fireplace=True)</i>


<u><b>One-hot variables</b></u>:
Electrical, Heating, MSZoning, Neighborhood, LandContour, LotConfig, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, Foundation, GarageType


####  Adding value-based features

In [11]:
train = MyValueAddedFeatures().fit_transform(train)

#### Adding quality/conditions related features

In [12]:
train = MyQualityFeatures().fit_transform(train)

#### Adding time-based features

In [13]:
train = MyTimeBasedFeatures().fit_transform(train)

#### Adding rooms-based features

In [14]:
train = MyRoomsFeatures().fit_transform(train)

#### Adding space-based features

In [15]:
train = MySpaceBasedFeatures().fit_transform(train)

#### Applying Log transformation on all features with skewness > 0.75

In [16]:
train = MyLog1pTransformer().fit_transform(train)

#### Initial Pipeline with all numerical features and categorical "Neighborhood" and new "season"

In [ ]:
cols_to_drop = ['MSSubClass', 'Id', 'Utilities', 'Street', 'MasVnrArea', 'Condition1', 'Condition2']
cols_enc_quality = ['BsmtCond', 'BsmtQual', 'ExterCond', 'ExterQual', 'FireplaceQu', 
                'GarageCond', 'GarageQual', 'HeatingQC', 'KitchenQual', 'PoolQC']
cols_enc_ordinal = ['BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageFinish',
                'LotShape', 'PavedDrive', 'Electrical', 'Functional', 'HouseStyle',
                   'LandSlope']
cols_enc_binary = ['Alley', 'BldgType', 'MiscFeature', 'Fence', 'MasVnrType']

cols_log = ['1stFlrSF', 'LotArea', 'LotFrontage', 'GrLivArea', 'BsmtUnfSF', 'TotalBsmtSF']

cols_1hot = ['Heating', 'MSZoning', 'LandContour',
                    'LotConfig', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
                    'Foundation', 'GarageType', 'CentralAir', 'SaleType', 'SaleCondition']


preprocessing = Pipeline([
    ('drop_cols', MyDropColumns(cols_to_drop)),
    ('quality_encoder', MyQualityEncoder(cols_enc_quality)),
    ('order_encoder', MyOtherOrdinalEncoder(cols_enc_ordinal)),
    ('binary_encoder', MyBinaryEncoder(cols_enc_binary)),
    ('imputer', MySimpleImputer()),
    ('drop_1hot', MyDropColumns(cols_1hot)) 
])

feature_creation = Pipeline([
    ('value', MyValueAddedFeatures()),
    ('quality', MyQualityFeatures()),
    ('time', MyTimeBasedFeatures()),
    ('rooms', MyRoomsFeatures()),
    ('space', MySpaceBasedFeatures()),
    ('new_log_transformer', MyLog1pTransformer()),
    ('onehot', MyDummyFeatures()), # dummy for Neighborhood and Season
])

full_pipeline = Pipeline([
    ('cleaning', preprocessing),
    ('features', feature_creation),
    ('median_scaler', RobustScaler()),
    ('lasso_cv', LassoCV(cv=5))
])

X = original_train.drop('SalePrice',axis=1)
y = np.log(original_train.SalePrice) # predicting log(SalePrice)

full_pipeline.fit(X,y)

y_pred = full_pipeline.predict(X)

rmsle = np.sqrt(mse(y,y_pred))
print(rmsle)

#y_test = full_pipeline.predict(test)
#submission = pd.DataFrame({'Id': test.Id, 'SalePrice': np.exp(y_test)})
#submission.to_csv('submission.csv', index=False)

## 3.2 feature selection

### a. filtering

Given the regression problem, I will filter features based on Spearman correlation. 

In [22]:
correlated_features = abs(train.corr(method='spearman')['SalePrice_log'].sort_values(ascending=False))>0.5
correlated_w_target = train.corr(method='spearman')['SalePrice_log'].sort_values(ascending=False)[correlated_features].index
train[correlated_w_target].corr(method='spearman')

,SalePrice,SalePrice_log,OverallQual,GrLivArea,GrLivArea_log,TotBath,GarageCars,ExterQual,ExterQual_log,BsmtQual,...,FireplaceQu,FireplacesValue,FireplacesValue_log,TotRmsAbvGrd,Fireplaces,Rooms_vs_LivArea,Bedrooms_vs_LivArea,YrsSinceBuilt,YrsSinceRemod,HighQualSF_percent
SalePrice,1.000000,1.000000,0.809829,0.731310,0.731310,0.703731,0.690711,0.684014,0.684014,0.678026,...,0.537602,0.535240,0.535240,0.532586,0.519247,-0.591402,-0.603423,-0.650120,-0.650120,-0.698218
SalePrice_log,1.000000,1.000000,0.809829,0.731310,0.731310,0.703731,0.690711,0.684014,0.684014,0.678026,...,0.537602,0.535240,0.535240,0.532586,0.519247,-0.591402,-0.603423,-0.650120,-0.650120,-0.698218
OverallQual,0.809829,0.809829,1.000000,0.603262,0.603262,0.557466,0.608756,0.715988,0.715988,0.673048,...,0.481197,0.457004,0.457004,0.427806,0.420626,-0.515645,-0.564124,-0.644136,-0.644136,-0.585324
GrLivArea,0.731310,0.731310,0.603262,1.000000,1.000000,0.604521,0.505094,0.446430,0.446430,0.388268,...,0.480506,0.490210,0.490210,0.827874,0.480804,-0.660436,-0.592034,-0.289056,-0.289056,-0.975395
GrLivArea_log,0.731310,0.731310,0.603262,1.000000,1.000000,0.604521,0.505094,0.446430,0.446430,0.388268,...,0.480506,0.490210,0.490210,0.827874,0.480804,-0.660436,-0.592034,-0.289056,-0.289056,-0.975395
TotBath,0.703731,0.703731,0.557466,0.604521,0.604521,1.000000,0.529361,0.494642,0.494642,0.562690,...,0.311082,0.323720,0.323720,0.453237,0.344650,-0.478045,-0.463544,-0.551269,-0.551269,-0.581665
GarageCars,0.690711,0.690711,0.608756,0.505094,0.505094,0.529361,1.000000,0.542498,0.542498,0.551884,...,0.357182,0.338349,0.338349,0.386244,0.325520,-0.399061,-0.466648,-0.599968,-0.599968,-0.486020
ExterQual,0.684014,0.684014,0.715988,0.446430,0.446430,0.494642,0.542498,1.000000,1.000000,0.645766,...,0.352144,0.312949,0.312949,0.300065,0.262436,-0.412697,-0.504639,-0.676205,-0.676205,-0.431857
ExterQual_log,0.684014,0.684014,0.715988,0.446430,0.446430,0.494642,0.542498,1.000000,1.000000,0.645766,...,0.352144,0.312949,0.312949,0.300065,0.262436,-0.412697,-0.504639,-0.676205,-0.676205,-0.431857
BsmtQual,0.678026,0.678026,0.673048,0.388268,0.388268,0.562690,0.551884,0.645766,0.645766,1.000000,...,0.317001,0.283493,0.283493,0.224725,0.262486,-0.403366,-0.478718,-0.772184,-0.772184,-0.370589


### b. wrapper

Given the large number of features, I decided not to use wrapper methods as it might take significant time to figuring out the best combination of features and it is not guaranteed to give me the best model. 

### c. embedded methods

Let's run Lasso for final feature selection. 

# 3. Final pipeline